In [1]:
from qiskit import IBMQ

In [ ]:
IBMQ.load_account()

In [4]:
import numpy as np
import networkx as nx
from operator import itemgetter
import import_ipynb
import QLS_for_GB2_mod as qls

importing Jupyter notebook from QLS_for_GB2_mod.ipynb
importing Jupyter notebook from QAOA_for_QLS.ipynb
importing Jupyter notebook from Side_Calc_GB2.ipynb


In [5]:
def compute_mod(G,k): #returns modularity for given consideration of node attributes
    #--> alternative approaches to yield maximum modularity: brute force search in space
    #of power set with detected communities (by recursive_QLS) as its elements
    for i in G.nodes:
        sum=0
        for j in range(0,k):
            sum=sum+pow(2,j)*G.nodes[i]["Spin{}".format(j)] #binary encoding of community by node attributes
        G.nodes[i]["colour"]=sum
    
    groups = set(nx.get_node_attributes(G,'colour').values())
    communities=list()
    
    for l in groups: #create propper community form for nx-function
        group=set()
        for i in G.nodes:
            if G.nodes[i]["colour"]==l:
                group.add(i)
        communities.append(group)
    
    result=nx.algorithms.community.quality.modularity(G, communities)
    return result, communities

In [6]:
def recursive_QLS(G,k, backend='simulator', subset_size=12, iter=0):
    if len(G)==0:
        return 0 #stopping criteria if there are no further nodes
    lis=list(G.nodes())
    n=len(lis)
    subgraph2graph=dict((x,lis[x]) for x in range (n))
    graph2subgraph=dict((lis[x],x) for x in range (n))
    
    result=qls.QLS(G, backend, subset_size, method='QAOA') #call QLS for partition into two communities
    solution=result[2]
    iter_time=(result[0])
    #global total_time
    #total_time+=iter_time
    
    if -1 in solution:
        solution = [0 if x == -1 else 1 if x == 1 else 'Error' for x in solution]
        
    
    for i,j in zip(G.nodes, range(0,len(G))): #add node attributes according to QLS solution
        G.nodes[i]["Spin{}".format(iter)]=solution[j]
        
    #create subgraphs for further recursions    
    subnodes1 = (node
                    for node, data
                    in G.nodes(data=True)
                    if data.get("Spin{}".format(iter)) == 1)
        
    subnodes2 = (node
                    for node, data
                    in G.nodes(data=True)
                    if data.get("Spin{}".format(iter)) == 0)
        
    G1 = G.subgraph(subnodes1)
    G2 = G.subgraph(subnodes2)
    
    
    iter+=1
    if iter>=k:
        return 0
    
    recursive_QLS(G1,k,backend, subset_size, iter)
    recursive_QLS(G2,k,backend, subset_size, iter)

In [7]:
def GB2_mod(graphname, k=None, backend='simulator', subset_size=None): #overall function specifically for Zachary Karate Club Graph (generalization stated in code/paper)
    G=nx.read_gpickle(graphname)
    if k==None:
        k=int(np.log2(int(np.sqrt(len(G)))+1))+1
    #k=3 #for this special case: Zachary Karate Club has 34 nodes
    #--> number of communities c=int(sqrt(34))+1, --> k=int(log2(c))+1=3
    
    if subset_size==None: 
        provider = IBMQ.get_provider(hub='ibm-q')
        if backend=='melbourne':
            bd=provider.get_backend('ibmq_16_melbourne')
        elif backend=='simulator':
            bd=provider.get_backend('ibmq_qasm_simulator')
        else: bd=provider.get_backend(backend)
        
        prop=bd.properties()
        if prop==None:
            raise ValueError ("For some backends (e.g. the simulators), a subset_size has to be specified as there is no information about available qubits!")
        subset_size=len(prop.to_dict()['qubits'])-2 #minus two in case ancillary qubits are required
    
    #total_time=0
    
    recursive_QLS(G,k, backend, subset_size)
    
    mod0=[0,0]
    mod1=0
    
    for i in range (1,k):
        mod1=compute_mod(G,i) #found to be highest modularity when considering 4 communities
        
        if mod1[0]>mod0[0]:
            mod0=mod1
        
        
    communities=mod0[1]
    return (communities, mod0[0])#total_time)